In [ ]:
import pylabel
from pylabel import importer
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
day_path = 'voc_day/Annotations/'
night_path = 'voc_night/Annotations/'

In [ ]:
day_dataset = importer.ImportVOC(day_path)
night_dataset = importer.ImportVOC(night_path)

In [ ]:
day_df.img_folder = 'voc_day/JPEGImages/'
night_df.img_folder = 'voc_night/JPEGImages/'

In [ ]:
fullday_df = pd.concat([day_df, night_df], axis=0)

In [ ]:
processed_df = fullday_df[~fullday_df['cat_name'].isin(['Dog', 'Cow', 'Badger', 'Y.T.Marten'])]

In [ ]:
processed_df = processed_df.reset_index(drop=True)

In [ ]:
cats = processed_df.cat_name.value_counts().index

In [ ]:
cats_name = {}
number = 0
for cat in cats:
    cats_name[cat] = number
    number += 1

In [ ]:
for row in range(processed_df.shape[0]):
    processed_df.loc[row, 'cat_id'] = cats_name[processed_df.loc[row, 'cat_name']]

In [ ]:
processed_dataset = pylabel.dataset.Dataset(processed_df)

In [ ]:
splitter = pylabel.splitter.Split(processed_dataset)
splitter.StratifiedGroupShuffleSplit(
    train_pct=0.8,
    test_pct=0.1,
    val_pct=0.1,
    cat_col=['cat_name', 'img_folder'],
)

In [ ]:
splitter.dataset.export.ExportToYoloV5(use_splits=True, copy_images=True)

In [ ]:
splitted_df = splitter.dataset.df

train_df = splitted_df[splitted_df['split'] == 'train'].drop(columns='split')
test_df = splitted_df[splitted_df['split'] == 'test'].drop(columns='split')
valid_df = splitted_df[splitted_df['split'] == 'val'].drop(columns='split')

In [ ]:
train_df.to_csv('train_df.csv')
test_df.to_csv('test_df.csv')
valid_df.to_csv('valid_df.csv')